In [34]:
import json
import random
import torch

from transformers import BertTokenizer, BertModel
from transformers import DistilBertTokenizer, DistilBertModel
from IPython.display import display

In [35]:
with open("../data/train.json") as json_file:
    data = json.load(json_file)

In [36]:
doc_num = random.randint(0,len(data)-1)
file_num = random.randint(0,len(data[doc_num]['annotations'][0]['result'])-1)
sentence = data[doc_num]['annotations'][0]['result'][file_num]['value']['text']
label = data[doc_num]['annotations'][0]['result'][file_num]['value']['labels']
print(f"Doc: {doc_num}, Sentence: {file_num}")
print(f"Sentence: {sentence}\nLabel: {label}")

Doc: 20, Sentence: 43
Sentence: 
 The High Court accepted the petitioners' plea that 30.3.85 is the date of first sale manufactured goods by the petitioners.
Label: ['FAC']


In [37]:
tokenizer1 = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)
tokenizer2 = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case = True)

In [38]:
inputs1 = tokenizer1(sentence, return_tensors="pt", truncation= True, padding= True, add_special_tokens= True)
inputs2 = tokenizer2(sentence, return_tensors="pt", truncation= True, padding= True, add_special_tokens= True)
display(inputs1)
display(inputs2)

{'input_ids': tensor([[  101,  1996,  2152,  2457,  3970,  1996,  9964,  2545,  1005, 14865,
          2008,  2382,  1012,  1017,  1012,  5594,  2003,  1996,  3058,  1997,
          2034,  5096,  7609,  5350,  2011,  1996,  9964,  2545,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]])}

{'input_ids': tensor([[  101,  1996,  2152,  2457,  3970,  1996,  9964,  2545,  1005, 14865,
          2008,  2382,  1012,  1017,  1012,  5594,  2003,  1996,  3058,  1997,
          2034,  5096,  7609,  5350,  2011,  1996,  9964,  2545,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]])}

In [39]:
model1 = BertModel.from_pretrained('bert-base-uncased')
model2 = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [40]:
bert_out = model1(**inputs1)
distbert_out = model2(**inputs2)

In [41]:
bert_out[0].size()

torch.Size([1, 30, 768])

In [42]:
distbert_out[0].size()

torch.Size([1, 30, 768])

In [60]:
emb = torch.nn.Embedding(30,50)

In [61]:
inputs1['input_ids']

tensor([[  101,  1996,  2152,  2457,  3970,  1996,  9964,  2545,  1005, 14865,
          2008,  2382,  1012,  1017,  1012,  5594,  2003,  1996,  3058,  1997,
          2034,  5096,  7609,  5350,  2011,  1996,  9964,  2545,  1012,   102]])

In [69]:
class SimpleLSTM(torch.nn.Module):
    def __init__(self,input_size, hidden_size, num_layers, output_size) -> None:
        super().__init__()
        
        self.hidden_size = hidden_size
        
        self.emb = torch.nn.Embedding(input_size, hidden_size)
        self.lstm = torch.nn.LSTM(
            input_size=hidden_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
        )
        self.dense = torch.nn.Sequential(
            torch.nn.Linear(in_features= hidden_size, out_features= 64, bias= True),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features= 64, out_features= output_size, bias= True),
            torch.nn.Softmax(),
        )
        
    def forward(self,x):
        output = self.emb(x)
        
        return output

In [70]:
simple_lstm = SimpleLSTM(30,128,1,13)

In [72]:
simple_lstm(inputs1['input_ids'].squeeze(0))

IndexError: index out of range in self

## Cant use bert ids and get own embeddings